# Data collector
#### For FERMI data from Oct/Nov 2017 at DiProI beamline

## Info

This script works on the raw HDF5 files saved by the DiProI DAQ.

The script collects and reduces the data from data sources such as XAS detector, XES spectrometer, delay stage and more .. (can be selected below) and saves the reduced data in a new HDF5 file that combines the data from all delay files within one photon energy folder.

Uses:
- /src/data_tools_v2.py
- /src/fit_tools.py


-----------------------
Created by Simon Schreck (simon.schreck@fysik.su.se) in Dec 2017 - Jan 2018 - with some parts beeing courtousy of Fivos Perakis.

Last updated: April 2018, by Simon Schreck (simon.schreck@fysik.su.se)

# 1. Initialization
#### Load packages and functions

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import h5py
import sys
import time
import os.path
from importlib import reload

sys.path.insert(0, '../src/')

# Blob finder needs to be set up on each system/computer specifically
# The 'import blobfinder' must be commented-out unless the blobfinder is set up on your system/computer
# import Blobfinder

import data_tools_v2
reload(data_tools_v2)

import fit_tools
reload(fit_tools)

import data_tools_v2 as dt
from fit_tools import *

##########################################
### DEFINE WHERE FIGURES ARE DISPLAYED ###
##########################################

# After changing this the kernel needs to be restarted and the cell needs to be run again
# Plot figures in in external windows (may need to set up your python environment properly for this to work...)
# %matplotlib qt
# Plot figures inline in notebook (works always!)
%matplotlib inline


# 2. Define Paramters

In [2]:
####################################
### DEFINE WHICH DATA TO COLLECT ###
####################################

def get_get_Params ():
    
    get_XAS_int        = True    # XES integrated intensity saved by DiProI DAQ (only good if thresholding was applied in DAQ already)
    get_XAS_image_int  = True    # Integrated intensity of XAS image (applies a threshold before integrating the image)
    get_XAS_image_blob = False   # MUST be False for now (unless Blobfinder is set up on your system)

    get_XES_int        = True    # XES integrated intensity saved by DiProI DAQ (only good if thresholding was applied in DAQ already)
    get_XES_image_int  = True    # Integrated intensity of XES image (applies a threshold before integrating the image)
    get_XES_image_blob = False   # MUST be False for now (unless Blobfinder is set up on your system)
    
    get_XES_spec       = True    # Keep Flase unless you really need the XES. (Increases runtime significanlty!)
    
    get_LASER_int      = True
    get_LASER_delay    = True

    get_MASSSPEC       = True

    get_i0             = True
    
    return get_XAS_int, get_XAS_image_int, get_XAS_image_blob, get_XES_int, get_XES_image_int, get_XES_image_blob, get_XES_spec, get_LASER_int, get_LASER_delay, get_MASSSPEC, get_i0


########################################
### DEFINE SOME ADDITIONAL PARAMETER ###
########################################

# Define cameras for XAS and XES
XAS_basler = 'BaslerImage2'
XES_basler = 'BaslerImage1'

# Pixel threshold for XAS and XES detector
xas_thr = 20
xes_thr = 20

# XES curvature file
def get_curv_file(bt):
    if bt == 2:
        curv_file = 'XES001_NoPump_curv.h5'
    else :
        curv_file = 'XAS002_curv.h5'
    return curv_file
    
# Blob fining parameters
clustersize     = 4
threshold       = 7


#############################
### DEBUGGING / TESTING ? ###
#############################

debug        = False
num_energies = 2 # MUST be at least 2
num_delays   = 3 # Should be at least 2


# 3. Do the Data Collection

In [5]:
# Reload data and fit tools
import data_tools_v2
reload(data_tools_v2)

import fit_tools
reload(fit_tools)

import data_tools_v2 as dt
from fit_tools import *

### --- START OF DEFINITION WHICH DATA TO COLLECT --- ###

################################
### DEFINE THE RUN(S) ##########
################################

# All XAS runs
#runs = [0, 99, 9, 17, 10, 23, 1, 3, 12, 13, 14, 18, 19, 20, 27, 24, 28, 29, 30, 31, 35, 36]

#runs = [31,32,33,34,36]
runs = [1,3,12,13,5,6,9,10]

# All XES runs
#runs = [1,2,3,4,5,6,7,8,9,11,12,13,14,15,16]

#runs = [15, 16]

#################################
### DEFINE IF XAS or XES RUN ####
#################################

run_type = 'XAS' # 'XAS' or 'XES'

################################
### DEFINE THE BEAMTIME ########
################################

BT = 2 # 1: 1st beamtime, 2: 2nd beamtime

################################
### DEFINE THE DATA PATH #######
################################

if(BT==1):
    data_path = 'D:/FERMI 2017 1/'
else:
    data_path = 'D:/FERMI 2017 2/'
    
### --- END OF DEFINITION WHICH DATA TO COLLECT --- ###

### --- START OF MAIN SCRIPT --- ###
if debug :
    print( 'debug is TRUE!!!')
    print('--> Will only run over %d energy folder(s) and %d delay file(s) per folder.\n' %(num_energies - 1, num_delays + 1) )
    print('Will NOT save the data!')
    
# Start timer
t = time.time()

# Check that data path exists
if not os.path.exists(data_path) :
    raise NameError('Check data_path! It does not exist!')

# Check that all runs exist
i = 0
while i < len(runs) :
    # Create the path to the run folder
    run_path = dt.do_runpath(runs[i], run_type, BT, data_path)
    
    if not os.path.exists(run_path) :
        print('Run %s%03d does not exist! Skipping this run!'%(run_type, runs[i]))
        runs = np.delete(runs, i)
    else :
        i = i + 1
        
print('Start collecting runs:')
print(runs)
        
# Loop over runs
for run in runs :
    # Create the path to the run folder
    run_path = dt.do_runpath(run, run_type, BT, data_path)
    print(run_path)
    
    # Get all photon energy folder names in run folder
    folders,tmp = dt.discover_files(run_path)
    
    # Get the FEL harmonic for this run and beamtime
    harm = dt.getHarm(run, run_type, BT)
    
    # Get polarization (read it from folder name)
    if not folders[0]=='combined' :
        pol_ind = 0
    else :
        pol_ind = 1
    if folders[pol_ind][-3:] == 'Ver' :
        polarization = 1
    elif folders[pol_ind][-3:] == 'Hor' :
        polarization = 0
    else :
        polarization = 99
    
    # Loop over photon energy folders
    f_c = 1
    for i in range(len(folders)):
        
        # Skip the 'combined' folder
        if folders[i]=='combined': continue
        
        # If debugging / testing: do only first num_energies folders
        if debug == True and f_c == num_energies : break
            
        print('Collecting data from folder %s (%d of %d)' %(folders[i], f_c, len(folders)-1))
        f_c = f_c +1
        
        # Path to data in this folder
        load_path = run_path+folders[i]+'/rawdata/'

        # Get all delay file names in this photon energy folder
        tmp,file_names = dt.discover_files(load_path)

        # Get get parameters (defined in 'Parameters' cell above)
        get_XAS_int, get_XAS_image_int, get_XAS_image_blob, get_XES_int, get_XES_image_int, get_XES_image_blob, get_XES_spec, get_LASER_int, get_LASER_delay, get_MASSSPEC, get_i0 = get_get_Params()

        # Check if data exist
        fh5_check = h5py.File(load_path+file_names[0], 'r')
               
        if get_XAS_int :
            if not '/Laser/BaslerInt2' in fh5_check:
                print('XAS INTENSITY is missing in folder %s. Skipping XAS INTENSITY in folder %s!' %(folders[i], folders[i]))
                get_XAS_int = False
        
        if get_XAS_image_int or get_XAS_image_blob :
            if not '/Laser/'+XAS_basler in fh5_check:
                print('XAS Basler is missing in folder %s. Skipping XAS Basler in folder %s!' %(folders[i], folders[i]))
                get_XAS_image_int  = False
                get_XAS_image_blob = False
                
        if get_XES_image_int or get_XES_image_blob or get_XES_spec:
            if not '/Laser/'+XES_basler in fh5_check:
                print('XES Basler is missing in folder %s. Skipping XES Basler in folder %s!' %(folders[i], folders[i]))
                get_XES_image_int  = False
                get_XES_image_blob = False
                get_XES_spec       = False
                
        if get_XES_int :
            if not '/Laser/BaslerInt1' in fh5_check:
                print('XES INTENSITY is missing in folder %s. Skipping XES INTENSITY in folder %s!' %(folders[i], folders[i]))
                get_XES_int = False
                
        if get_LASER_int :
            if not '/Laser/Energy1' in fh5_check:
                print('LASER INTENSITY is missing in folder %s. Skipping LASER INTENSITY in folder %s!' %(folders[i], folders[i]))
                get_LASER_int = False
                
        if get_LASER_delay :
            if not '/Laser/DelayPosVector' in fh5_check:
                print('LASER DELAY is missing in folder %s. Skipping LASER DELAY in folder %s!' %(folders[i], folders[i]))
                get_LASER_delay = False 

        if get_MASSSPEC:
            if not '/Lecroy/Wave1' in fh5_check:
                print('MASS SPEC is missing in folder %s. Skipping MASS SPEC in folder %s!' %(folders[i], folders[i]))
                get_MASSSPEC = False 
        
        if get_i0 :
            if not '/photon_diagnostics/Spectrometer/hor_spectrum' in fh5_check:
                print('I0 SPECTROMETER is missing in folder %s. Skipping I0 SPECTROMETER in folder %s!' %(folders[i], folders[i]))
                get_i0 = False            

        # Get time zero used in that run
        if f_c == 2 :
            comment = fh5_check['ExperimentalComments'].value
            t0_str = '0'
            for s in np.arange(len(comment)) :
                if comment[s] == '=' :
                    t0_str = comment[s+1:s+8]                    
            t0_num = float(t0_str)
            
        # Close the 'check' hdf5 file
        fh5_check.close()
        
        # Define empty arrays to collect data in
        bunch_id_all         = []
        xas_int_all          = []
        xas_tfy_all          = []
        xas_tfy_blobsnum_all = []
        xes_blobs_x_all      = []
        xes_blobs_y_all      = []
        xes_blobsnum_all     = []
        xes_int_all          = []
        xes_pfy_all          = []
        xes_spec_all         = []
        delay_pos_all        = []
        laser_int_all        = []
        ms_tof_all           = []
        ms_counts_all        = []
        
        ########################
        ### COLLECT THE DATA ###
        ########################
        
        # Loop over delay files in photon energy folder
        for j in range(len(file_names)):
            # If debugging / testing: do only first num_delays delay files
            if debug == True and j > num_delays : break
            
            print(file_names[j])
            try :
                h5file = h5py.File(load_path+file_names[j], 'r')
            except IOError :
                print('File ' + file_names[j] + ' could not be read. Skipping File!')
                continue
                
            # Bunch ID
            bunch_id      = h5file['/bunches'].value
            bunch_id_all.extend(bunch_id)
            
            # XAS TFY intensity from file
            if get_XAS_int :
                xas_int = h5file['/Laser/BaslerInt2'].value
                xas_int_all.extend(xas_int)
            # XAS TFY intensity from image
            if get_XAS_image_int :
                #xas_tfy = dt.get_XAS_intensity(h5file, xas_thr)
                xas_tfy = dt.get_Basler_intensity(h5file, XAS_basler, xas_thr)
                xas_tfy_all.extend(xas_tfy)
            # XAS TFY blobs
            if get_XAS_image_blob :
                xas_blobs_x, xas_blobs_y, xas_tfy_blobsnum = dt.get_Basler_blobs(h5file, XAS_basler, clustersize, threshold)
                xas_tfy_blobsnum_all.extend(xas_tfy_blobsnum)
            
            # XES
            if get_XES_image_blob :
                xes_blobs_x, xes_blobs_y, xes_blobsnum = dt.get_Basler_blobs(h5file, XES_basler, clustersize, threshold)
                xes_blobs_x_all.extend(xes_blobs_x)
                xes_blobs_y_all.extend(xes_blobs_y)
                xes_blobsnum_all.extend(xes_blobsnum)
            # XES intensity from file
            if get_XES_int :
                xes_int = h5file['/Laser/BaslerInt1'].value
                xes_int_all.extend(xes_int)
            # XES TFY intensity from image
            if get_XES_image_int :
                xes_pfy = dt.get_Basler_intensity(h5file, XES_basler, xes_thr)
                xes_pfy_all.extend(xes_pfy)
            # XES spectra from image
            if get_XES_spec :
                curv_file = get_curv_file(BT)
#                curv_file = data_path + 'Curvatures/' + curv_file
                curv_file = '../src/' + curv_file
                xes_spec = dt.get_Basler_projection(h5file, XES_basler, curv_file, xes_thr)
                xes_spec_all.extend(xes_spec)
                                    
            # LASER
            if get_LASER_int :
                laser_int = h5file['/Laser/Energy1'].value
                laser_int_all.extend(laser_int)
            if get_LASER_delay :    
                delay_pos = h5file['/Laser/DelayPosVector'].value
                delay_pos_all.extend(delay_pos)
            
            # MassSpec
            if get_MASSSPEC :
                ms_tof, ms_counts = dt.get_ms(h5file)
                ms_tof_all.extend(ms_tof)
                ms_counts_all.extend(ms_counts)
            
            # End of loop over delay files
            
        # I0 and FEL statistics for all delay files (this function has an internal loop over all delay files)        
        if get_i0:
            # Get i0 and FEL statistics and average FEL spectrum for all delay files in photon energy folder
            i0, FEL_eV, FEL_Avg_Spectrum, Gauss_amps, Gauss_centers, Gauss_widths, fitfail_counter = dt.get_i0(file_names, load_path, harm, Peak_Width = 50, Bcg_Width = 10, get_FELstats = True, debug = debug, num_delays = num_delays)
        
        # Close the hdf5 file
        h5file.close()
        
        
        if not debug :
            
            #####################################################
            ### SAVE THE DATA (for this photon energy folder) ###
            #####################################################
            
            # Define the filename for saved data
            run_str = '%s%03d'%(run_type, run)
            save_file   = run_path+'combined/'+run_str+'_'+folders[i]+'_col.h5'            

            # Open file (if exists), else create file)
            fh5 = h5py.File(save_file, 'a') 

            # Bunch ID
            if 'bunch_id' in fh5: del fh5['bunch_id']
            dataSet = fh5.create_dataset('bunch_id', data = bunch_id_all, dtype = np.int32)
            # Polarization
            if 'polarization' in fh5: del fh5['polarization']
            dataSet = fh5.create_dataset('polarization', data = polarization, dtype = np.int32)
            # Time zero
            if 't0' in fh5: del fh5['t0']
            dataSet = fh5.create_dataset('t0', data = t0_num, dtype = np.float32)
            if 'harmonic' in fh5: del fh5['harmonic']
            dataSet = fh5.create_dataset('harmonic', data = harm, dtype = np.float32)

            # XAS TFY intensity
            if get_XAS_image_int :
                if 'XAS/xas_tfy' in fh5: del fh5['XAS/xas_tfy']
                dataSet = fh5.create_dataset('XAS/xas_tfy', data = xas_tfy_all, dtype = np.int32)
                if 'XAS/xas_thr' in fh5: del fh5['XAS/xas_thr']
                dataSet = fh5.create_dataset('XAS/xas_thr', data = xas_thr, dtype = np.int32)
            if get_XAS_int :
                if 'XAS/xas_int' in fh5: del fh5['XAS/xas_int']
                dataSet = fh5.create_dataset('XAS/xas_int', data = xas_int_all, dtype = np.int32)
            # XAS TFY blobs
            if get_XAS_image_blob :
                if 'XAS/xas_tfy_blobsnum' in fh5: del fh5['XAS/xas_tfy_blobsnum']
                dataSet = fh5.create_dataset('XAS/xas_tfy_blobsnum', data = xas_tfy_blobsnum_all, dtype = np.int32)
                if 'XAS/clustersize' in fh5: del fh5['XAS/clustersize']
                dataSet = fh5.create_dataset('XAS/clustersize', data = clustersize, dtype = np.int32)
                if 'XAS/threshold' in fh5: del fh5['XAS/threshold']
                dataSet = fh5.create_dataset('XAS/threshold', data = threshold, dtype = np.int32)
            # XES
            if get_XES_image_int :
                if 'XES/xes_pfy' in fh5: del fh5['XES/xes_pfy']
                dataSet = fh5.create_dataset('XES/xes_pfy', data = xes_pfy_all, dtype = np.int32)
                if 'XES/xes_thr' in fh5: del fh5['XES/xes_thr']
                dataSet = fh5.create_dataset('XES/xes_thr', data = xes_thr, dtype = np.int32)
            if get_XES_image_blob :
                if 'XES/xes_blobsnum' in fh5: del fh5['XES/xes_blobsnum']
                dataSet = fh5.create_dataset('XES/xes_blobsnum', data = xes_blobsnum_all, dtype = np.int32)
                if 'XES/xes_blobs_x' in fh5: del fh5['XES/xes_blobs_x']
                dataSet = fh5.create_dataset('XES/xes_blobs_x', data = xes_blobs_x_all, dtype = np.float64)
                if 'XES/xes_blobs_y' in fh5: del fh5['XES/xes_blobs_y']
                dataSet = fh5.create_dataset('XES/xes_blobs_y', data = xes_blobs_y_all, dtype = np.float64)
                if 'XAS/clustersize' in fh5: del fh5['XAS/clustersize']
                dataSet = fh5.create_dataset('XAS/clustersize', data = clustersize, dtype = np.int32)
                if 'XAS/threshold' in fh5: del fh5['XAS/threshold']
                dataSet = fh5.create_dataset('XAS/threshold', data = threshold, dtype = np.int32)
                
            if get_XES_spec :
                if 'XES/xes_spec' in fh5: del fh5['XES/xes_spec']
                dataSet = fh5.create_dataset('XES/xes_spec', data = xes_spec_all, dtype = np.float64)
                
            if get_XES_int :
                if 'XES/xes_int' in fh5: del fh5['XES/xes_int']
                dataSet = fh5.create_dataset('XES/xes_int', data = xes_int_all, dtype = np.int32)
            # LASER
            if get_LASER_delay :
                if 'LASER/delay_pos' in fh5: del fh5['LASER/delay_pos']
                dataSet = fh5.create_dataset('LASER/delay_pos', data = delay_pos_all, dtype = np.float64)
            if get_LASER_int :
                if 'LASER/laser_int' in fh5: del fh5['LASER/laser_int']
                dataSet = fh5.create_dataset('LASER/laser_int', data = laser_int_all, dtype = np.float64)
            # MassSpec
            if get_MASSSPEC :
                if 'MASSSPEC/tof' in fh5: del fh5['MASSSPEC/tof']
                dataSet = fh5.create_dataset('MASSSPEC/tof', data = ms_tof_all, dtype = np.float64)
                if 'MASSSPEC/counts' in fh5: del fh5['MASSSPEC/counts']
                dataSet = fh5.create_dataset('MASSSPEC/counts', data = ms_counts_all, dtype = np.int64)
            # I0 and FEL statistics
            if get_i0:
                if 'FEL/i0' in fh5: del fh5['FEL/i0']
                dataSet = fh5.create_dataset('FEL/i0', data = i0, dtype = np.float64)
                if 'FEL/FEL_eV' in fh5: del fh5['FEL/FEL_eV']
                dataSet = fh5.create_dataset('FEL/FEL_eV', data = FEL_eV, dtype = np.float64)
                if 'FEL/FEL_Avg_Spectrum' in fh5: del fh5['FEL/FEL_Avg_Spectrum']
                dataSet = fh5.create_dataset('FEL/FEL_Avg_Spectrum', data = FEL_Avg_Spectrum, dtype = np.float64)
                if 'FEL/Gauss_amps' in fh5: del fh5['FEL/Gauss_amps']
                dataSet = fh5.create_dataset('FEL/Gauss_amps', data = Gauss_amps, dtype = np.float64)
                if 'FEL/Gauss_centers' in fh5: del fh5['FEL/Gauss_centers']
                dataSet = fh5.create_dataset('FEL/Gauss_centers', data = Gauss_centers, dtype = np.float64)
                if 'FEL/Gauss_widths' in fh5: del fh5['FEL/Gauss_widths']
                dataSet = fh5.create_dataset('FEL/Gauss_widths', data = Gauss_widths, dtype = np.float64)
                if 'FEL/fitfail_counter' in fh5: del fh5['FEL/fitfail_counter']
                dataSet = fh5.create_dataset('FEL/fitfail_counter', data = fitfail_counter, dtype = np.int32)

            fh5.close() # Close the file
            print('Saved file '+run_str+'_'+folders[i]+'_col.h5')
        else:
            print('debug = True --> Data were not saved!')
        
        # End of loop over photon energy folders
    
    # End of loop over runs

print('\nDONE!')
print('Total time elapsed (in sec):')
print(time.time() - t)

plt.show()


Start collecting runs:
[1, 3, 12, 13, 5, 6, 9, 10]
D:/FERMI 2017 2/XAS001/
XES Basler is missing in folder E285p50eV_Ver. Skipping XES Basler in folder E285p50eV_Ver!
XES INTENSITY is missing in folder E285p50eV_Ver. Skipping XES INTENSITY in folder E285p50eV_Ver!
MASS SPEC is missing in folder E285p50eV_Ver. Skipping MASS SPEC in folder E285p50eV_Ver!
E285p50eV_Ver_367660832.h5


<ipython-input-5-eec3a7de7a74>:173: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  comment = fh5_check['ExperimentalComments'].value
<ipython-input-5-eec3a7de7a74>:216: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  bunch_id      = h5file['/bunches'].value
<ipython-input-5-eec3a7de7a74>:221: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  xas_int = h5file['/Laser/BaslerInt2'].value
<ipython-input-5-eec3a7de7a74>:257: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  laser_int = h5file['/Laser/Energy1'].value
<ipython-input-5-eec3a7de7a74>:260: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  delay_pos = h5file['/Laser/DelayPosVector'].value


E285p50eV_Ver_367661244.h5
E285p50eV_Ver_367661545.h5
E285p50eV_Ver_367661851.h5
E285p50eV_Ver_367662145.h5
E285p50eV_Ver_367662451.h5
E285p50eV_Ver_367662775.h5
E285p50eV_Ver_367663070.h5
E285p50eV_Ver_367663375.h5
E285p50eV_Ver_367663680.h5
E285p50eV_Ver_367663999.h5
E285p50eV_Ver_367664303.h5
E285p50eV_Ver_367664606.h5
E285p50eV_Ver_367664898.h5
E285p50eV_Ver_367665201.h5
E285p50eV_Ver_367665504.h5
E285p50eV_Ver_367665800.h5
E285p50eV_Ver_367666104.h5
E285p50eV_Ver_367666399.h5
E285p50eV_Ver_367666703.h5
E285p50eV_Ver_367666999.h5
E285p50eV_Ver_367667302.h5
E285p50eV_Ver_367667598.h5
E285p50eV_Ver_367667892.h5
E285p50eV_Ver_367668224.h5
E285p50eV_Ver_367668528.h5
E285p50eV_Ver_367668852.h5
E285p50eV_Ver_367669155.h5
E285p50eV_Ver_367669478.h5
E285p50eV_Ver_367669843.h5
E285p50eV_Ver_367670292.h5
E285p50eV_Ver_367670835.h5
Saved file XAS001_E285p50eV_Ver_col.h5
E286p00eV_Ver_367673228.h5


<ipython-input-5-eec3a7de7a74>:241: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  xes_int = h5file['/Laser/BaslerInt1'].value


E286p00eV_Ver_367673584.h5
E286p00eV_Ver_367673919.h5
E286p00eV_Ver_367674247.h5
E286p00eV_Ver_367674572.h5
E286p00eV_Ver_367674901.h5
E286p00eV_Ver_367675237.h5
E286p00eV_Ver_367675561.h5
E286p00eV_Ver_367675893.h5
E286p00eV_Ver_367676222.h5
E286p00eV_Ver_367676548.h5
E286p00eV_Ver_367676882.h5
E286p00eV_Ver_367677211.h5
E286p00eV_Ver_367677540.h5
E286p00eV_Ver_367677869.h5
E286p00eV_Ver_367678198.h5
E286p00eV_Ver_367678522.h5
E286p00eV_Ver_367678859.h5
E286p00eV_Ver_367679188.h5
E286p00eV_Ver_367680568.h5
E286p00eV_Ver_367680895.h5
E286p00eV_Ver_367681221.h5
E286p00eV_Ver_367681549.h5
E286p00eV_Ver_367681871.h5
E286p00eV_Ver_367682927.h5
E286p00eV_Ver_367683257.h5
E286p00eV_Ver_367683591.h5
E286p00eV_Ver_367683920.h5
E286p00eV_Ver_367684280.h5
E286p00eV_Ver_367684634.h5
E286p00eV_Ver_367685113.h5
E286p00eV_Ver_367685651.h5
Saved file XAS001_E286p00eV_Ver_col.h5
E286p29eV_Ver_367688501.h5
E286p29eV_Ver_367688854.h5
E286p29eV_Ver_367689189.h5
E286p29eV_Ver_367689514.h5
E286p29eV_Ver_36

../src\fit_tools.py:45: RuntimeWarning: invalid value encountered in sqrt
  perr = np.sqrt(np.diag(pcov))


Saved file XAS003_E285p50eV_Ver_col.h5
E286p00eV_Ver_368312386.h5
E286p00eV_Ver_368312840.h5
E286p00eV_Ver_368313284.h5
E286p00eV_Ver_368313714.h5
E286p00eV_Ver_368314133.h5
E286p00eV_Ver_368314543.h5
E286p00eV_Ver_368314996.h5
E286p00eV_Ver_368315408.h5
E286p00eV_Ver_368315826.h5
E286p00eV_Ver_368316246.h5
E286p00eV_Ver_368316660.h5
E286p00eV_Ver_368317080.h5
E286p00eV_Ver_368317533.h5
E286p00eV_Ver_368317958.h5
E286p00eV_Ver_368318374.h5
E286p00eV_Ver_368318816.h5
E286p00eV_Ver_368319243.h5
E286p00eV_Ver_368319654.h5
E286p00eV_Ver_368320082.h5
E286p00eV_Ver_368320502.h5
E286p00eV_Ver_368320921.h5
E286p00eV_Ver_368321363.h5
E286p00eV_Ver_368321818.h5
E286p00eV_Ver_368322225.h5
E286p00eV_Ver_368322623.h5
E286p00eV_Ver_368323049.h5
E286p00eV_Ver_368323468.h5
E286p00eV_Ver_368323886.h5
E286p00eV_Ver_368324332.h5
E286p00eV_Ver_368324813.h5
E286p00eV_Ver_368325380.h5
E286p00eV_Ver_368326009.h5
Saved file XAS003_E286p00eV_Ver_col.h5
E286p29eV_Ver_368293376.h5
E286p29eV_Ver_368293800.h5
E286

### If having problems with open HDF5 files, run below lines to close all HDF5 files that are still open

#### This can happen when e.g. the script crashes and files stay open.

In [8]:
import gc
for obj in gc.get_objects():   # Browse through ALL objects
    if isinstance(obj, h5py.File):   # Just HDF5 files
        try:
            print('Closed %s.'%obj.filename)
            obj.close()
        except:
            pass # Was already closed
print('Done!')

AttributeError: 'functools._lru_list_elem' object has no attribute '__class__'